In [25]:
import os
from src.utils import load_yaml, get_event_urls, get_fighter_urls, get_fight_urls, convert_models_to_dicts
from src.scrapers import scrape_events, scrape_fighters, scrape_rounds, scrape_results
from pathlib import Path
import json

In [ ]:
root_dir = os.path.abspath('.')
config_path = os.path.join(root_dir, 'config', 'config.yaml')
config = load_yaml(config_path)

 

In [ ]:
events = scrape_events(config['event_urls']['all'])
event_urls = get_event_urls(events_data=[row.dict() for row in events])

In [ ]:
results = scrape_results(event_urls=event_urls)
fighter_urls = get_fighter_urls(results_data=[row.dict() for row in results])
fight_urls = get_fight_urls(results_data=[row.dict() for row in results])
fighters = scrape_fighters(fighter_urls=fighter_urls)
rounds = scrape_rounds(fight_urls=fight_urls)



NameError: name 'get_fighter_urls' is not defined

In [27]:

def save_models_to_json(models, filepath: str) -> None:
    """
    Save a list of Pydantic models to a JSON file.

    Args:
        models (List[BaseModel]): List of Pydantic model instances.
        filepath (str): The path to the JSON file where the data will be saved.
    """
    # Convert each model to a dict
    data = convert_models_to_dicts(models=models)
    
    # Ensure the directory exists
    path = Path(filepath)
    path.parent.mkdir(parents=True, exist_ok=True)
    
    # Write the list of dictionaries to the JSON file
    with path.open("w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [28]:
config

{'output_files': {'events': 'data/ufc_events.json',
  'results': 'data/ufc_results.json',
  'fighters': 'data/ufc_fighters.json',
  'rounds': 'data/ufc_rounds.json'},
 'event_urls': {'all': 'http://ufcstats.com/statistics/events/completed?page=all',
  'one': 'http://ufcstats.com/statistics/events/completed?page=1'}}

In [38]:
save_models_to_json(
    models=events, 
    filepath=os.path.join(root_dir, config['output_files']['raw']['events'])
    )

In [39]:
save_models_to_json(
    models=fighters, 
    filepath=os.path.join(root_dir, config['output_files']['raw']['fighters'])
    )

In [40]:
save_models_to_json(
    models=rounds, 
    filepath=os.path.join(root_dir, config['output_files']['raw']['rounds'])
    )

In [41]:
save_models_to_json(
    models=results, 
    filepath=os.path.join(root_dir, config['output_files']['raw']['results'])
    )